# <center> **Base de données (Films)**

## **Présentation**

Des fichiers csv contenant les données ont été générés lors de la phase précédente de scrapping, nous allons maintenant lire ces fichiers csv dans des dataFrame Pandas et nettoyer les données puis les insérer dans une base de données SQL.


## **Questions**

**Quelles données mettre en SQL et en NoSQL ?**<br>
SQL pour les données tabulaires, NoSQL pour les données non tabulaires ou non structurées.

**Générer automatiquement des IDs chaines de caractères ?**

**Les requêtes faites à partir de notre API doivent avoir accès uniquement à la base en lecture seule**


## **Sources**

**Neo4j pour créer des graphs ML**<br>
https://neo4j.com/docs/getting-started/appendix/tutorials/guide-import-relational-and-etl/


**Wikidata**<br>
https://query.wikidata.org/querybuilder/?uselang=fr<br>


## **Création du schéma relationnel**

![relation_schema](images/DB_diagram.png)

Online tool: https://drawdb.vercel.app/editor<br>
https://www.mocodo.net/<br>

L'utilitaire en ligne **drawdb** permet d'exporter un fichier .SQL contenant la description de notre base (tables avec les relations)

![export_SQL](images/export_SQL.png)

Après quelques modifications à ce fichier nous pouvons créer notre base de données ainsi que les tables à partir de l'invite de commandes.

<code>"C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code>



Une difficulté rencontrée est de faire apparaître les films similaires à un film dans la base de données, en effet cette situation est une relation "Many to many" où la table se référence elle-même, c'est un cas fréquent dans les réseaux sociaux où un utilisateur va avoir des amis eux-mêmes utilisateurs.<br>
Deux situations sont observées : <br>
- les relations symétriques (l'utilisateur Alice est amis avec l'utilisateur Bob implique nécessairement que Bob est ami d'Alice),
- les relations asymétriques (Alice est amis avec Bob mais Bob n'est pas forcément ami avec Alice).

Dans notre situation la relation est .....????

Pour représenter cette relation ....





**About Self many-to-many relationship**: https://stackoverflow.com/questions/17128472/many-to-many-on-same-table

Relations n-n

Remarque :
La table "reviews" sert de table de jonction entre les films et les utilisateurs, en effet la relation movies-users est une relation n-n car un utilateur peut écrire des avis pour plusieurs films et un film possède des avis de plusieurs utilisateurs.




## **Création de la base de données**

On lance **MySQL Shell** puis on passe en mode **SQ** avec l'instruction <code>\sql</code>

On peut ensuite lancer la création de notre base de données et des tables en lançant le script "movies.sql"

<code> "C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < movies.sql -u root -p</code><br>

Une fois la base 'movies' créée, on peut créer un connecteur sur la base.

In [49]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='admin', \
                              host = '127.0.0.1', database='movies')
cursor = cnx.cursor(buffered=True)

In [160]:
cnx.disconnect()

## **Remplissage de la base de données**

### **Imports**

In [85]:
import math
import copy
import time
import re
import uuid
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display

pd.set_option('display.max_rows', 10)
tqdm.pandas()

### **Tools**

In [ ]:
def string_list_to_list_of_strings(st):
    if '[]' in st:
        st = st[1:-1]
    return [item.strip() for item in st.split("'") if len(item.strip()) > 1]

# txt = "['dario argento', 'joe d\'amato']"
# txt = '["Joe D\'Amato"]'
txt = '["Pat O\'Connor"]'
# txt = '["Dan O\'Bannon"]'
# txt = "['Anthony Hopkins', 'John Hurt', 'Jack Nicholson', 'Shelley Duvall']"
# txt = "['David Lynch']"
print(string_list_to_list_of_strings(txt))

In [162]:
def string_with_comma_to_list_of_strings(st):
    ''' Convert a string such as "['string1', 'string2' ...]" into
        a list of string ['string1', 'string2', ...]

        Return: A list of strings.

        Arg:
         - st: string with the value to split.
    '''
    if pd.isna(st):
        return []
    return [item.strip() for item in st.split(",") if len(item.strip()) > 1]

def fill_in_table_from_list(lst, table_name, field_id, field, connector, cursor):
    ''' Fill in an SQL table from a list of values.
        For each value in the list 'lst' we generate an ID and insert into the table (ID, value).
        CAREFUL: To run only once for each table, otherwise : "ERROR Duplicate entry '0' for key 'table_name.PRIMARY'"

        Return: A dictionary mapping each value of the list to an ID newly generated {value1 : ID1, value2 : ID2, ...}

        Args:
         - lst: list of values to insert into the table,
         - table_name: string with the name of the table,
         - field_id: string with the ID of the value record inserted in the table,
         - field: string of the field in the table,
         - connector: MySQL connector connected to the relevant database.
    '''
    dic_return = {}
    for item in lst:
        dic_return[item] = str(uuid.uuid4())
    #     sql = f"INSERT INTO {table_name} ({field_id}, {field}) VALUES (%s, %s)"
    #     val = (dic_return[item], item)
    #     cursor.execute(sql, val)
    # connector.commit()
    return dic_return

def unique_values_from_columns(df_data, column):
    ''' return a list with unique values found in the column,
        values in the column are like : 'value1,value2,value3' ..... 
        so we split all values in each row of the column and stack them in a list.

        Return : A list of values.

        Args:
         - df_data: dataframe with the data to extract,
         - column: string with the name of the column we want to work with.     
    '''
    df = df_data[column]
    df.dropna(inplace = True)
    df = df.apply(string_with_comma_to_list_of_strings)
    df = df.apply(pd.Series).stack().reset_index(drop=True)
    return df.unique()

### **Lecture des données à partir des CSV**

In [161]:
# ds_categories = pd.read_csv('csv/categories.csv', delimiter = ',')
# ds_countries = pd.read_csv('csv/countries.csv', delimiter = ',')
arr_categories = np.load('csv/categories.npy')
arr_countries = np.load('csv/countries.npy')
# df_movies = pd.read_csv('csv/movies_year_1980.csv', delimiter = ',')
df_movies = pd.read_csv('csv/movies_year_1981.csv', delimiter = ',')
# df_movies = pd.read_csv('csv/movies_decade_80.csv', delimiter = ',')

print("Categories :", arr_categories.shape)
print("Countries :", arr_countries.shape)
print("movies :", df_movies.shape)

Categories : (37,)
Countries : (101,)
movies : (80, 16)


### **Remplissage des tables**

In [ ]:
# ------------------------------ #
#    Fill in categories table    #
# ------------------------------ #
dict_category_id = fill_in_table_from_list(arr_categories.tolist(), 'categories', 'category_id', 'category', cnx, cursor)

# ------------------------------ #
#    Fill in countries table     #
# ------------------------------ #
dict_country_id = fill_in_table_from_list(arr_countries.tolist(), 'countries', 'country_id', 'country', cnx, cursor)

# ------------------------------ #
#    Fill in directors table     #
# ------------------------------ #
lst_directors = unique_values_from_columns(df_movies, 'directors')
dict_director_id = fill_in_table_from_list(lst_directors, 'directors', 'director_id', 'director_name', cnx, cursor)

# ------------------------------ #
#      Fill in actors table      #
# ------------------------------ #
lst_actors = unique_values_from_columns(df_movies, 'actors')
dict_actor_id = fill_in_table_from_list(lst_actors, 'actors', 'actor_id', 'actor_name', cnx, cursor)

# ------------------------------ #
#     Fill in composers table    #
# ------------------------------ #
lst_composers = unique_values_from_columns(df_movies, 'composers')
dict_composer_id = fill_in_table_from_list(lst_composers, 'composers', 'composer_id', 'composer_name', cnx, cursor)

In [145]:
list(dict_director_id.keys())

['Jean Girault',
 'Georges Lautner',
 'Steven Spielberg',
 'Pierre Granier-Deferre',
 'Michael Mann',
 'Uli Edel',
 'Ulu Grosbard',
 'Andrzej Zulawski',
 'François Truffaut',
 'Francis Veber',
 'Robert Hossein',
 'Jean-Jacques Annaud',
 'Jamie Uys',
 'Brian De Palma',
 'John Carpenter',
 'John Boorman',
 'George Miller',
 'Wolfgang Petersen',
 'Claude Lelouch',
 'Claude Miller',
 'Bertrand Tavernier',
 'Sam Raimi',
 'Yves Boisset',
 'Desmond Davis',
 'Hugh Hudson',
 'Patrice Leconte',
 'John Landis',
 'Jean-Jacques Beineix',
 'Walter Hill',
 'Bob Rafelson',
 'David Cronenberg',
 'Claude Berri',
 'Joe Dante',
 'René Laloux',
 'Lawrence Kasdan',
 'Noel Marshall',
 'Jean-Marie Poiré',
 'Mark Rydell',
 'Alain Corneau',
 'Karel Reisz',
 'John Huston',
 'André Téchiné',
 'Abel Ferrara',
 'Mel Brooks',
 'Alain Delon',
 'François Leterrier',
 'John Glen',
 'Umberto Lenzi',
 'Michel Deville',
 'Milos Forman',
 'Peter Weir',
 'Michael Wadleigh',
 'Hal Needham',
 'Terry Gilliam',
 'Sidney Lumet',

### **Remplissage de la table des films**

In [156]:
# --------------------------------------------------------
#
# Converting some columns of "df_movies" into the appropriate format:
#   - correct date format
#   - duration un minutes
#   - IDs of the categories
#   - IDs of the directors / actors / composers
#
# --------------------------------------------------------

df_movies_formatted = df_movies.copy()
df_movies_formatted['categories'] = df_movies_formatted['categories'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['categories'] = df_movies_formatted['categories'].apply(lambda lst : [dict_category_id[k] for k in lst])

df_movies_formatted['countries'] = df_movies_formatted['countries'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['countries'] = df_movies_formatted['countries'].apply(lambda lst : [dict_country_id[k] for k in lst])

df_movies_formatted['directors'] = df_movies_formatted['directors'].map(string_with_comma_to_list_of_strings)
df_movies_formatted['directors'] = df_movies_formatted['directors'].apply(lambda lst : [dict_director_id[k] for k in lst])

df_movies_formatted['actors'] = df_movies_formatted['actors'].apply(string_with_comma_to_list_of_strings)
df_movies_formatted['actors'] = df_movies_formatted['actors'].apply(lambda lst : [dict_actor_id[k] for k in lst])

df_movies_formatted['composers'] = df_movies_formatted['composers'].apply(string_with_comma_to_list_of_strings)
df_movies_formatted['composers'] = df_movies_formatted['composers'].apply(lambda lst : [dict_composer_id[k] for k in lst])

KeyError: 'Botswana'

In [80]:
df_movies.head(5)

,Unnamed: 0,title,date,duration,categories,countries,star_rating,notes,critics,directors,actors,composers,summary,url_thumbnail,url_reviews,url_similar_movies
0,0,Elephant Man,8 avril 1981,2h 05min,"Biopic,Drame",U.S.A.,"4,4",29356,689,['David Lynch'],"['Anthony Hopkins', 'John Hurt', 'Anne Bancrof...",['John Morris (II)'],"Londres, 1884. Le chirurgien Frederick Treves ...",https://fr.web.img4.acsta.net/c_310_420/pictur...,https://www.allocine.fr/film/fichefilm-180/cri...,https://www.allocine.fr/film/fichefilm-180/sim...
1,1,Cannibal Holocaust,22 avril 1981,1h 26min,Epouvante-horreur,"Italie,Colombie","2,1",5795,548,['Ruggero Deodato'],"['Robert Kerman', 'Francesca Ciardi', 'Perry P...",['Riz Ortolani'],Une équipe de journalistes composée de trois h...,https://fr.web.img3.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-52677/c...,https://www.allocine.fr/film/fichefilm-52677/s...
2,2,Shining,16 octobre 1980,2h 23min,"Epouvante-horreur,Thriller","Grande-Bretagne,U.S.A.","4,3",59203,1753,['Stanley Kubrick'],"['Jack Nicholson', 'Shelley Duvall', 'Danny Ll...","['Wendy Carlos', 'Rachel Elkind']","Écrivain, Jack Torrance est engagé comme gardi...",https://fr.web.img3.acsta.net/c_310_420/pictur...,https://www.allocine.fr/film/fichefilm-863/cri...,https://www.allocine.fr/film/fichefilm-863/sim...
3,3,La Boum,17 décembre 1980,1h 49min,"Comédie,Drame,Romance",France,"3,0",18586,227,['Claude Pinoteau'],"['Sophie Marceau', 'Brigitte Fossey', 'Claude ...",['Vladimir Cosma'],"Vic vit tranquillement entre le lycée, ses par...",https://fr.web.img3.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-4403/cr...,https://www.allocine.fr/film/fichefilm-4403/si...
4,4,Beau-Père,16 septembre 1981,2h 03min,Drame,France,"3,6",821,82,['Bertrand Blier'],"['Patrick Dewaere', 'Ariel Besse', 'Maurice Ro...",['Philippe Sarde'],"Rémi est un musicien à la dérive. Sa compagne,...",https://fr.web.img3.acsta.net/c_310_420/pictur...,https://www.allocine.fr/film/fichefilm-1403/cr...,https://www.allocine.fr/film/fichefilm-1403/si...


In [157]:
df_movies_formatted

,Unnamed: 0,title,date,duration,categories,countries,star_rating,notes,critics,directors,actors,composers,summary,url_thumbnail,url_reviews,url_similar_movies
0,0,La Soupe aux choux,2 décembre 1981,1h 38min,"[742a768b-ec03-401a-8a43-7e5187bfad0b, cd5b564...",[France],"3,3",25697,428,Jean Girault,"Louis de Funès,Jean Carmet,Claude Gensac,Marco...",Raymond Lefevre,"Le Glaude et le Bombé, deux vieux paysans port...",https://fr.web.img6.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-30750/c...,https://www.allocine.fr/film/fichefilm-30750/s...
1,1,Le Professionnel,21 octobre 1981,1h 45min,"[a8a5350f-765d-4965-88d2-84ff17480901, 19f5759...",[France],"3,7",11042,267,Georges Lautner,"Jean-Paul Belmondo,Jean Desailly,Robert Hossei...",Ennio Morricone,"Issu de l'élite de l'armée française, Joss Bea...",https://fr.web.img3.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-43249/c...,https://www.allocine.fr/film/fichefilm-43249/s...
2,2,Les Aventuriers de l'Arche perdue,16 septembre 1981,1h 56min,"[a8a5350f-765d-4965-88d2-84ff17480901, ba0eed9...",[U.S.A.],"4,4",69947,960,Steven Spielberg,"Harrison Ford,Karen Allen,Paul Freeman,John Rh...",John Williams,1936. Parti à la recherche d'une idole sacrée ...,https://fr.web.img2.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-121/cri...,https://www.allocine.fr/film/fichefilm-121/sim...
3,3,Une étrange affaire,23 décembre 1981,1h 45min,[773f985c-6bf1-4f3c-8dbb-83456cd6a9a9],[France],"3,6",377,37,Pierre Granier-Deferre,"Michel Piccoli,Gérard Lanvin,Nathalie Baye,Jea...",Philippe Sarde,"Louis, jeune publicitaire, se voit confier de ...",https://fr.web.img2.acsta.net/c_310_420/pictur...,https://www.allocine.fr/film/fichefilm-4690/cr...,https://www.allocine.fr/film/fichefilm-4690/si...
4,4,Le Solitaire,19 mai 1981,2h 02min,"[19f57597-4505-4acc-a686-06502512ad96, 9c05c62...",[U.S.A.],"3,9",858,95,Michael Mann,"James Caan,Tuesday Weld,James Belushi,Robert P...","Edgar Froese,Christopher Franke,Johannes Schmö...","Après onze ans passés en prison, Frank, un tal...",https://fr.web.img2.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-41777/c...,https://www.allocine.fr/film/fichefilm-41777/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,Le Dragon du lac de feu,20 octobre 1982,1h 48min,"[ba0eed90-20b1-44a7-8787-a4c46eff26f8, 66a98a8...",[U.S.A.],"3,4",176,25,Matthew Robbins,"Peter MacNicol,Caitlin Clarke,Ralph Richardson...",Alex North,"Sous la menace d'un terrible dragon, un Roi dé...",https://fr.web.img6.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-60579/c...,https://www.allocine.fr/film/fichefilm-60579/s...
76,76,Massacres dans le train fantôme,19 mai 20,1h 36min,"[c7a7b21e-21cc-4909-9d8a-acd3f921173a, edda528...",[U.S.A.],"2,5",199,44,Tobe Hooper,"Elizabeth Berridge,Shawn Carson,Jeanne Austin,...",John Beal,C'est la fête ! Un parc d'attraction vient de ...,https://fr.web.img4.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-111146/...,https://www.allocine.fr/film/fichefilm-111146/...
77,77,Les Bleus,31 mars 1982,1h 46min,[742a768b-ec03-401a-8a43-7e5187bfad0b],[U.S.A.],"2,3",182,21,Ivan Reitman,"Bill Murray,Harold Ramis,Warren Oates,P.J. Sol...",Elmer Bernstein,"Le même jour, John Winger perd son travail, sa...",https://fr.web.img6.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-30744/c...,https://www.allocine.fr/film/fichefilm-30744/s...
78,78,"Salut l'ami, adieu le trésor",16 décembre 1981,1h 48min,"[a8a5350f-765d-4965-88d2-84ff17480901, ba0eed9...","[U.S.A., Italie]","3,0",271,32,Sergio Corbucci,"Terence Hill,Bud Spencer,John Fujioka,Sal Borg...","Carmelo La Bionda,Michelangelo La Bionda",Alan et son ami Charlie partent à la recherche...,https://fr.web.img6.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-58949/c...,https://www.allocine.fr/film/fichefilm-58949/s...


In [159]:
df_movies[df_movies['countries'].str.contains('Botswana')]

,Unnamed: 0,title,date,duration,categories,countries,star_rating,notes,critics,directors,actors,composers,summary,url_thumbnail,url_reviews,url_similar_movies
12,12,Les Dieux sont tombés sur la tête,19 janvier 1983,1h 40min,Comédie,Botswana,"3,5",1971,93,Jamie Uys,"Marius Weyers,Sandra Prinsloo,N'Xau,Jamie Uys,...",NaN,"Une bouteile de coca-cola, jetee d'un avion, a...",https://fr.web.img2.acsta.net/c_310_420/medias...,https://www.allocine.fr/film/fichefilm-2577/cr...,https://www.allocine.fr/film/fichefilm-2577/si...
